In [3]:
import numpy as np
import math
from IPython.core.debugger import set_trace

lowerBound, upperBound = -30, 30
x = np.random.uniform(low=lowerBound, high=upperBound, size=(10,))
N = len(x)

def f(x):
    #Sphere
    Sum = 0
    for i in range(0,N):
        Sum = Sum + (x[i])**2

    #Rosenbrock
    #Sum = 0
    #for i in range(0, N-1):
    #    Sum = Sum + 100 * ((x[i])**2 - x[i+1])**2 + (x[i] - 1)**2
        
    #Ackley
    #Sum1, Sum2 = 0, 0
    #for i in range(0,N):
    #    Sum1 = Sum1 + (x[i])**2
    #    Sum2 = Sum2 + math.cos(2*math.pi*x[i])
    #Sum= -20.0*math.exp(-0.2*math.sqrt((1/N)*(Sum1)))-math.exp((1/N) * Sum2) + math.e + 20
    return Sum

l = [lowerBound for i in range(10)]
u = [upperBound for i in range(10)]

In [4]:
def _e(index,size):
    arr = np.zeros(size)
    arr[index] = 1.0
    return arr

In [5]:
def Delta(x):
    Sum = 0
    #no general, only box constraints
    for i in range(0,N):
        Sum = Sum + max(x[i]-u[i],0) + max(l[i]-x[i], 0)
    return Sum

In [6]:
def better(x,y):
    Dx = Delta(x)
    Dy = Delta(y)
    if(Dx < Dy):
        return True
    elif(Dx == Dy and f(x) < f(y)):
        return True
    else:
        return False

In [7]:
def findExtremes(s):
    Ds = np.empty([N+1])
    fs = np.empty([N+1])
    
    for i in range(0, N+1):
        Ds[i] = Delta(s[i])
        fs[i] = f(s[i])
    
    arg_b = np.argmin(Ds)
    for i in range(0, N+1):
        if Ds[i] == Ds[arg_b] and i != arg_b:
            if(fs[i] < fs[arg_b]):
                arg_b = i;
                
    arg_w = np.argmax(Ds)
    for i in range(0, N+1):
        if Ds[i] == Ds[arg_w] and i != arg_w:
            if(fs[i] > fs[arg_w]):
                arg_w = i;
    
    #mask the worst to find the second worst
    Ds[arg_w] = 0
    
    arg_sw = np.argmax(Ds)
    for i in range(0, N+1):
        if Ds[i] == Ds[arg_sw] and i != arg_w:
            if(fs[i] > fs[arg_sw]):
                arg_sw = i;
    
    return arg_b,arg_w,arg_sw

In [8]:
def centroid(s,argWorst):
    return np.mean(np.delete(s, argWorst), axis = 0)

In [9]:
def SimplexLocalSearch(Epsilon,Lambda,M,Alpha=1,Gamma=2,Beta=-0.5):
    s = np.empty([N+1,N])
    s[0] = x
    for j in range(1,N+1):
        s[j] = x + Lambda*(u[j-1]-l[j-1])*_e(j-1,N);
    k = 0
    
    
    argBest, argWorst, argSecondWorst = findExtremes(s)
    sBest, sWorst, sSecondWorst = s[argBest], s[argWorst], s[argSecondWorst]
    #set_trace()
    while abs(f(sBest) - f(sWorst)) > Epsilon and k < M:
        #set_trace()
        if(k%1000 == 0):print(f(sBest),'\n')
        r = (1+Alpha)*centroid(s,argWorst) - Alpha*sWorst
        if(better(sBest,r) and better(r,sSecondWorst)):
            s[argWorst] = r
        elif(better(r,sBest)):
            e = (1+Gamma)*centroid(s,argWorst)-Gamma*sWorst
            if(better(e,r)):
                s[argWorst] = e
            else:
                s[argWorst] = r
        else:
            c = (1+Beta)*centroid(s,argWorst)-Beta*sWorst
            if(better(c,sWorst)):
                s[argWorst] = c
            else:
                for j in range(0,N+1):
                    s[j] = (s[j] + sBest)/2

        #set_trace()
        argBest, argWorst, argSecondWorst = findExtremes(s)
        sBest, sWorst, sSecondWorst = s[argBest], s[argWorst], s[argSecondWorst]
        k = k + 1
    print(f(sBest))
    return sBest

In [10]:
def Simplex(Epsilon, Lambda, M):
    '''read problem data'''
    xBest = SimplexLocalSearch(Epsilon, Lambda,M);
    return xBest

In [11]:
Lambda = 1
M = 100000
Epsilon = 0
print(Simplex(Epsilon,Lambda,M))

1401.3950833608483 

0.5977650635078252 

6.44870488676884e-05 

2.165551063201487e-11 

3.0251747879696887e-16 

2.8082229848020792e-24 

1.1707093578156776e-53 

4.6639332448066944e-58 

6.608754043434724e-61 

3.0065154826234606e-74 

5.593399865857576e-99 

8.651319932845578e-127 

1.016713272758106e-144 

2.427516506712041e-148 

4.693630788605555e-152 

9.162005731587388e-162 

1.5185782955071927e-166 

2.315870339130673e-170 

2.2176058585378704e-175 

7.641919914266127e-194 

1.6058467723907516e-233 

2.0080638520451772e-238 

1.0346698853426616e-247 

4.909161181002673e-261 

3.6965334417134657e-269 

4.36805741455331e-272 

1.959237389646171e-274 

7.004629339906071e-278 

1.0000972553274466e-284 

1.9501825033986416e-291 

9.06354e-319 

0.0
[-4.15650830e-163  8.99284275e-163 -1.13979463e-162  5.93152581e-163
  5.90300006e-163  8.77326987e-164  9.90422997e-163 -9.93177764e-163
  6.36773909e-163 -9.31140922e-165]
